In [1]:
import pandas as pd
import urllib
import requests
import random
import time
import numpy as np
import re

In [2]:
name_of_stock = ['GAP INC ', 'LEVI STRAUSS & CO', 'RALPH LAUREN CORP ', 'PVH CORP']

In [3]:
links = []
for name in name_of_stock:
    link = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company=' + name.replace(" ","+") + '&type=10-K&dateb=&owner=exclude&count=100'
    links.append(link)

In [4]:
links

['https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company=GAP+INC+&type=10-K&dateb=&owner=exclude&count=100',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company=LEVI+STRAUSS+&+CO&type=10-K&dateb=&owner=exclude&count=100',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company=RALPH+LAUREN+CORP+&type=10-K&dateb=&owner=exclude&count=100',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company=PVH+CORP&type=10-K&dateb=&owner=exclude&count=100']

In [5]:
headers_list =  [
        # Firefox 77 Mac
        {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
        },
        # Chrome 92.0 Win10
        {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
        },
        # Chrome 91.0 Win10
        {
        "Connection": "keep-alive",
        "DNT": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
        },
        # Firefox 90.0 Win10
        {
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9"
        }
    ]

In [6]:
substring = 'id="documentsbutton'
filing_type = []
date = []
company_name = []
doc_link_list = []
counter = 0
counter_a = 0
weblink = []

for link in links:
    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers
    time.sleep(np.random.uniform(2, 4))
    html = r.get(link).text
    counter += 1
    print(counter)
    if substring not in html:
        index1 = html.find('<table class="tableFile2" summary="Results">')
        html2 = html[index1:]
        start_index = html2.find("<a href=")
        html3 = html2[start_index:]
        href_start_index = html3.find("<a href=") + 9
        end_index = html3.find("\">")
        href = "https://www.sec.gov" + html3[href_start_index:end_index]
        href = href.replace("amp;", "")
        headers = random.choice(headers_list)
        r = requests.Session()
        r.headers = headers
        html_a = r.get(href).text
        company_start = html_a.find('<span class="companyName">') + len('<span class="companyName">')
        company_end = html_a.find('<acronym title="Central Index Key">')
        company = html_a[company_start:company_end]
        start = html_a.find("<table class=\"tableFile2\" summary=\"Results\">")
        html_a_2 = html_a[start:]
        end = html_a_2.find("</table>")
        html_a_3 = html_a_2[:end]
        rows = html_a_3.split("</tr>")
        for row in rows[1:len(rows) - 1]:
            row = row.replace('<td nowrap="nowrap">', '').strip()
            row = row.replace(' class="blueRow"', '')
            row = row.replace("\n", "")
            start_type = row.find("<tr>") + len("<tr>")
            end_type = row.find("</td>")
            filing_type.append(row[start_type:end_type])
            start_link = row.find('<a href="/') + len('<a href="/')
            end_link = row.find('" id=\"documentsbutton\">')
            link = "https://www.sec.gov/" + row[start_link:end_link]
            weblink.append(link)
            date_pattern = r'<td>(\d{4}-\d{2}-\d{2})</td>'
            match = re.search(date_pattern, row)
            if match:
                filing_date = match.group(1)
            date.append(filing_date)
            company_name.append(company)
    else:
        company_start = html.find('<span class="companyName">') + len('<span class="companyName">')
        company_end = html.find('<acronym title="Central Index Key">')
        company = html[company_start:company_end]
        start = html.find("<table class=\"tableFile2\" summary=\"Results\">")
        html_2 = html[start:]
        end = html_2.find("</table>")
        html_3 = html_2[:end]
        rows = html_3.split("</tr>")
        for row in rows[1:len(rows) - 1]:
            row = row.replace('<td nowrap="nowrap">', '').strip()
            row = row.replace(' class="blueRow"', '')
            row = row.replace("\n", "")
            start_type = row.find("<tr>") + len("<tr>")
            end_type = row.find("</td>")
            filing_type.append(row[start_type:end_type])
            start_link = row.find('<a href="/') + len('<a href="/')
            end_link = row.find('" id=\"documentsbutton\">')
            link = "https://www.sec.gov/" + row[start_link:end_link]
            weblink.append(link)
            date_pattern = r'<td>(\d{4}-\d{2}-\d{2})</td>'
            match = re.search(date_pattern, row)
            if match:
                filing_date = match.group(1)
            date.append(filing_date)
            company_name.append(company)
            
for w_link in weblink:
    counter_a += 1
    print(counter_a)
    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers
    time.sleep(np.random.uniform(2, 4))
    doc_html = r.get(w_link).text
    doc_start_index = doc_html.find("<td scope=\"row\">1</td>")
    doc_html_1 = doc_html[doc_start_index:]
    doc_start_index_2 = doc_html_1.find("<a href=\"") +len('<a href="')
    doc_html_2 = doc_html_1[doc_start_index_2:]
    doc_end = doc_html_2.find('">')
    doc_link = "https://www.sec.gov/" + doc_html_2[:doc_end]
    doc_link_list.append(doc_link)
        
data_frame = {
    "Company Name": company_name,
    "Date of Filing": date,
    "Filing Type": filing_type,
    "Link to the Document": doc_link_list
    
}
df = pd.DataFrame(data_frame)
df.to_csv('Webdata Group 10K Links.csv')

1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
